# Example data processing
This example demonstrates how a data engineer utilizes OEA to work with data from a new data source.

In [1]:
%run OEA_py

StatementMeta(, 6, -1, Finished, Available)

2022-11-17 19:47:11,995 - OEA - INFO - Now using workspace: dev
2022-11-17 19:47:11,996 - OEA - INFO - OEA initialized.


In [2]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace('sam')

StatementMeta(spark3p2sm, 6, 2, Finished, Available)

2022-11-17 19:53:00,151 - OEA - INFO - Now using workspace: sam


In [4]:
# 2) Now land a batch data file into stage1 of the data lake.
# In this example we pull a test csv data file from github and it is landed in oea/sandboxes/sam/stage1/Transactional/contoso/v0.1/students/delta_batch_data/rundate=<utc datetime>
data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day1/students/part1.csv').text
oea.land(data, 'contoso/v0.1/students', 'students.csv', oea.DELTA_BATCH_DATA)

StatementMeta(spark3p2sm, 6, 4, Finished, Available)

'stage1/Transactional/contoso/v0.1/students/delta_batch_data/rundate=2022-11-17 19:55:15/students.csv'

In [5]:
# 3) You can verify that the data is in stage1 by reading it into a dataframe. Note that a "rundate" column has been added - representing the datetime that the batch data was landed in the data lake.
df = oea.load_csv(f'stage1/Transactional/contoso/v0.1/students')
display(df)

StatementMeta(spark3p2sm, 6, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 81925c53-bee6-4d46-8900-2d01e60b5524)

In [6]:
# 4) The next step is to ingest the batch data into stage2
# Note that when you run this the first time, you'll see an info message like "Number of new inbound rows processed: 2".
# If you run this a second time, the number of inbound rows processed will be 0 because the ingestion uses spark structured streaming to keep track of what data has already been processed.
oea.ingest(f'contoso/v0.1/students', 'SIS ID')

StatementMeta(spark3p2sm, 6, 6, Finished, Available)

2022-11-17 19:56:37,527 - OEA - INFO - Ingesting from: stage1/Transactional/contoso/v0.1/students, batch type of: delta, source data format of: csv
2022-11-17 19:56:50,412 - py4j.java_gateway - INFO - Callback Server Starting
2022-11-17 19:56:50,414 - py4j.java_gateway - INFO - Socket listening on ('127.0.0.1', 36439)
2022-11-17 19:56:52,908 - py4j.java_gateway - INFO - Callback Connection ready to receive messages
2022-11-17 19:56:52,909 - py4j.java_gateway - INFO - Received command c on object id p0
2022-11-17 19:57:14,881 - OEA - INFO - Number of new inbound rows processed: 2


2

In [8]:
# 5) When data is ingested into stage2 of the data lake, OEA creates a lake db (which is a logical db that points to the data in the data lake).
# In this example, since you are working in the 'sam' workspace, the lake db created is called 'ldb_sam_s2i_contoso_v0p1' (if you click on Data in the left nav, you'll see the db listed under 'Lake database' )
df = spark.sql("select * from ldb_sam_s2i_contoso_v0p1.students")
display(df)
df.printSchema()

StatementMeta(spark3p2sm, 6, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3231fa94-b826-4ab8-93c1-8a4b6e845086)

root
 |-- SIS_ID: string (nullable = true)
 |-- School_SIS_ID: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- Middle_Name: string (nullable = true)
 |-- Secondary_Email: string (nullable = true)
 |-- Student_Number: string (nullable = true)
 |-- Grade: string (nullable = true)
 |-- State_ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Graduation_Year: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- FederalRaceCategory: string (nullable = true)
 |-- PrimaryLanguage: string (nullable = true)
 |-- ELLStatus: string (nullable = true)
 |-- SpecialEducation: string (nullable = true)
 |-- LowIncome: string (nullable = true)
 |-- CumulativeGPA: string (nullable = true)
 |-- rundate: timestamp (nullable = true)



In [10]:
# 6) Now let's land some additional inbound batch data - with new and modified rows.
data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day2/students/part1.csv').text
oea.land(data, 'contoso/v0.1/students', 'students.csv', oea.DELTA_BATCH_DATA)

StatementMeta(spark3p2sm, 6, 10, Finished, Available)

'stage1/Transactional/contoso/v0.1/students/delta_batch_data/rundate=2022-11-17 20:04:34/students.csv'

In [11]:
# 7) Ingest this latest batch of data.
# Note that you don't have to specify what batch of data to process; OEA uses spark structured streaming to determine what files are new.
oea.ingest(f'contoso/v0.1/students', 'SIS ID')

StatementMeta(spark3p2sm, 6, 11, Finished, Available)

2022-11-17 20:04:42,051 - OEA - INFO - Ingesting from: stage1/Transactional/contoso/v0.1/students, batch type of: delta, source data format of: csv
2022-11-17 20:04:44,272 - py4j.java_gateway - INFO - Callback Connection ready to receive messages
2022-11-17 20:04:44,273 - py4j.java_gateway - INFO - Received command c on object id p1
2022-11-17 20:04:54,676 - OEA - INFO - Number of new inbound rows processed: 2


2

In [12]:
# 8) Now verify that the batch data was ingested and correctly merged with the previous data

# You can load the ingested data into a dataframe directly like this...
df = oea.load('stage2/Ingested/contoso/v0.1/students')
display(df)

# ...or you can use the automatically created "Lake database" like this:
df = spark.sql("select * from ldb_sam_s2i_contoso_v0p1.students")
display(df)
# with either approach, you're querying the same data - it's the data stored at oea/sandboxes/sam/stage2/Ingested/contoso/v0.1/students in your data lake

StatementMeta(spark3p2sm, 6, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 500c7957-d473-4731-83a7-3e9834952376)

SynapseWidget(Synapse.DataFrame, 8710e716-0801-43f1-92fe-fdcbc726e3f1)

In [13]:
# 9) After ingesting data, the next step is to refine the data through the use of metadata
metadata = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/metadata.csv')
oea.refine('contoso/v0.1/students', metadata['students'], 'SIS ID')

StatementMeta(spark3p2sm, 6, 13, Finished, Available)

2022-11-17 20:05:50,416 - py4j.java_gateway - INFO - Received command  on object id 
2022-11-17 20:05:50,416 - py4j.java_gateway - INFO - Closing down callback connection
2022-11-17 20:05:56,971 - OEA - INFO - Processed 3 updated rows from stage2/Ingested/contoso/v0.1/students into stage2/Refined


3

In [15]:
# 12) Now you can query the refined data tables in the lake db
df = spark.sql("select * from ldb_sam_s2r_contoso_v0p1.students")
display(df)
df.printSchema()
df = spark.sql("select * from ldb_sam_s2r_contoso_v0p1.students_lookup")
display(df)
# You can use the "lookup" table for joins (people with restricted access won't be able to perform this query because they won't have access to data in the "sensitive" folder in the data lake)
df = spark.sql("select sl.Username, s.Grade from ldb_sam_s2r_contoso_v0p1.students_lookup sl, ldb_sam_s2r_contoso_v0p1.students s where sl.SIS_ID_pseudonym = s.SIS_ID_pseudonym")
display(df)

StatementMeta(spark3p2sm, 6, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 15bd05d2-ffe6-4954-9b7e-f08285a93f58)

root
 |-- SIS_ID_pseudonym: string (nullable = true)
 |-- School_SIS_ID: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Last_Name: string (nullable = true)
 |-- Middle_Name: string (nullable = true)
 |-- Secondary_Email: string (nullable = true)
 |-- Student_Number_pseudonym: string (nullable = true)
 |-- Grade: string (nullable = true)
 |-- State_ID_pseudonym: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Graduation_Year: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- FederalRaceCategory: string (nullable = true)
 |-- PrimaryLanguage: string (nullable = true)
 |-- ELLStatus: string (nullable = true)
 |-- SpecialEducation: string (nullable = true)
 |-- LowIncome: string (nullable = true)
 |-- CumulativeGPA: string (nullable = true)
 |-- rundate: timestamp (nullable = true)



SynapseWidget(Synapse.DataFrame, 6c971c59-9d28-4c2e-8680-07f3f38541a7)

SynapseWidget(Synapse.DataFrame, fbbddbf6-56b3-4d97-99e2-b4ef211ed6a2)

In [18]:
# 11) Land, ingest, and refine additional data sets.
# These data sets demonstrate the 2 other types of batch data - additive and snapshot.
data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day1/studentattendance/part1.csv').text
oea.land(data, 'contoso/v0.1/studentattendance', 'part1.csv', oea.ADDITIVE_BATCH_DATA)
data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day2/studentattendance/part1.csv').text
oea.land(data, 'contoso/v0.1/studentattendance', 'part1.csv', oea.ADDITIVE_BATCH_DATA)


data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day1/studentsectionmark/part1.csv').text
oea.land(data, 'contoso/v0.1/studentsectionmark', 'part1.csv', oea.SNAPSHOT_BATCH_DATA)
data = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/day2/studentsectionmark/part1.csv').text
oea.land(data, 'contoso/v0.1/studentsectionmark', 'part1.csv', oea.SNAPSHOT_BATCH_DATA)

oea.ingest(f'contoso/v0.1/studentattendance', 'id')
oea.ingest(f'contoso/v0.1/studentsectionmark', 'id')

metadata = requests.get('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/framework/test_data/contoso_sis/metadata.csv').text
oea.refine('contoso/v0.1/studentattendance', metadata['studentattendance'], 'id')
oea.refine('contoso/v0.1/studentsectionmark', metadata['studentsectionmark'], 'id')

StatementMeta(spark3p2sm, 6, 18, Finished, Available)

2022-11-17 20:18:31,463 - OEA - INFO - Ingesting from: stage1/Transactional/contoso/v0.1/studentattendance, batch type of: additive, source data format of: csv
2022-11-17 20:18:33,272 - py4j.java_gateway - INFO - Callback Connection ready to receive messages
2022-11-17 20:18:33,273 - py4j.java_gateway - INFO - Received command c on object id p4
2022-11-17 20:18:37,253 - OEA - INFO - Number of new inbound rows processed: 2928
2022-11-17 20:18:39,306 - OEA - INFO - Ingesting from: stage1/Transactional/contoso/v0.1/studentsectionmark, batch type of: snapshot, source data format of: csv
2022-11-17 20:18:40,953 - py4j.java_gateway - INFO - Received command c on object id p5
2022-11-17 20:18:43,814 - OEA - INFO - Number of new inbound rows processed: 24


TypeError: string indices must be integers

In [17]:
# 12) Reset this example
oea.delete_dataset('contoso/v0.1')
oea.drop_lake_db('ldb_sam_s2i_contoso_v0p1')
oea.drop_lake_db('ldb_sam_s2r_contoso_v0p1')

StatementMeta(spark3p2sm, 6, 17, Finished, Available)

2022-11-17 20:17:56,202 - OEA - INFO - Database dropped: ldb_sam_s2i_contoso_v0p1
2022-11-17 20:17:56,995 - OEA - INFO - Database dropped: ldb_sam_s2r_contoso_v0p1


'Database dropped: ldb_sam_s2r_contoso_v0p1'

# Appendix

In [ ]:
# You can list the contents of a folder in the data lake like this:
print(oea.ls('stage1/Transactional/contoso/v0.1/students/delta_batch_data'))
print(oea.ls('stage2/Ingested/contoso/v0.1/students'))

In [ ]:
dtbl = DeltaTable.forPath(spark, oea.to_url('stage2/Refined/contoso/v0.1/sensitive/students_lookup'))
display(dtbl.toDF())
#dtbl.delete("rundate > '2022-11-04T14:39:51'")